# Analisis de Audios con Gemini
En este Notebook, analizamos con ayuda de Gemini y Python los audios de las agencias de cobranza externa de la cartera castigada y vencida.
Necesitaremos acceso a nuestro Drive y a una "Key" de Gemini, con la cual se realizará el analísis.
<br>
<br>
Primero realizaremos unas pruebas para conectar a Drive y comprobar que tenemos permisos de edición sobre la carpeta:

In [ ]:
# 1. CONEXIÓN DE GOOGLE DRIVE
# Esto te pedirá que autorices de nuevo tu Drive.
from google.colab import drive
print("Intentando montar Google Drive...")
try:
    # Esto intentará montar /content/drive
    drive.mount('/content/drive', force_remount=True)
    print("✅ Google Drive montado exitosamente.")
except Exception as e:
    print(f"❌ Error al montar Drive: {e}")
    print("Por favor, verifica la consola para completar la autenticación.")



Intentando montar Google Drive...
Mounted at /content/drive
✅ Google Drive montado exitosamente.


In [ ]:

# 2. VERIFICAR LA EXISTENCIA DE LA CARPETA
# Reemplaza la ruta de ejemplo con la ruta donde intentas guardar/leer.

import os

#RUTA DE LAS GRABACIONES:
folder_to_check = '/content/drive/Shared drives/mi_ruta/'

print(f"\nVerificando si existe la carpeta: {folder_to_check}")

if os.path.exists(folder_to_check):
    print(f"✅ La ruta de la carpeta existe. Listando los primeros 10 archivos...")
    # Listar los primeros 10 elementos para confirmar el acceso
    files_in_folder = os.listdir(folder_to_check)
    for i, file in enumerate(files_in_folder):
        if i < 10:
            print(f"- {file}")
    if not files_in_folder:
        print("(Carpeta vacía)")

else:
    print(f"❌ ERROR: La carpeta NO existe en esta sesión: {folder_to_check}")
    print("Asegúrate de que la ruta sea exactamente la misma que usaste antes.")



Verificando si existe la carpeta: /content/drive/Shared drives/mi_ruta/
✅ La ruta de la carpeta existe. Listando los primeros 10 archivos...
- Empresa1
- an

In [ ]:

# 3.PRUEBA DE ESCRITURA/LECTURA SIMPLE
# Prueba a escribir y luego leer un archivo temporal en esa ruta para confirmar permisos.
test_file_path = os.path.join(folder_to_check, "TEST_ACCESS_GEMINI.txt")
test_content = "Acceso confirmado."

print(f"\nRealizando prueba de lectura/escritura en: {test_file_path}")

try:
    # Escribir
    with open(test_file_path, "w") as f:
        f.write(test_content)
    print("✅ Escritura exitosa.")

    # Leer
    with open(test_file_path, "r") as f:
        read_content = f.read()
    print(f"✅ Lectura exitosa. Contenido: '{read_content}'")

    # Limpiar
    os.remove(test_file_path)
    print("✅ Archivo de prueba eliminado.")

except FileNotFoundError:
    print("❌ ERROR: El camino del archivo no es correcto. Revisa la ruta.")
except PermissionError:
    print("❌ ERROR: No tienes permisos para escribir en esa ubicación de Drive.")
except Exception as e:
    print(f"❌ Ocurrió un error inesperado durante la prueba: {e}")


Realizando prueba de lectura/escritura en: /content/drive/Shared drives/mi_ruta/TEST_ACCESS_GEMINI.txt
✅ Escritura exitosa.
✅ Lectura exitosa. Contenido: 'Acceso confirmado.'
✅ Archivo de prueba eliminado.


Una vez que se configuro Drive, procederemos con el programa.
<br>
<br>
Primero comenzaremos con la configuración de Gemini y la carpeta donde estan los audios, es importante recalcar que cada mes se debe de cambiar y tambien debemos cambiar la ruta de la subcarpeta cuando notemos que hubo un error y solo necesitemos reprocesar una agencia, esto se hace en la variable <b> ´SUBFOLDER_TO_PROCESS´ </b>.
<br>
<br>
Tambien necesitamos configurar la ruta de los lineamientos cuando estos se actualicen, de momento no es necesario, pero esto se realiza en la variable "<b>RUTA_PROTECCION_DATOS</b>", "<b>RUTA_TRATO_CLIENTE</b>" y "<b>RUTA_CONDUSEF</b>".
<br>
<br>
Tambien es importante configurar la clave de tu gemini, de la API; para esto debemos de ir a la siguiente página: "https://aistudio.google.com/api-keys"; donde deberemos de crear una nueva API y pegarla en la variable "API_KEY"

Con esto se finaliza el proceso y debemos de verificar la creación del archivo txt en cada carpeta.



In [ ]:
import os
import glob
import unicodedata
import time
from google import genai
from google.colab import drive

# --- CONFIGURACIÓN DE RUTAS ---
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Configuración de API y Modelos
API_KEY = "Mi_API_KEY"
MODELO_PROCESO = "gemini-2.5-flash"
RUTA_BASE = "/content/drive/Shared drives/mi_ruta/Empresa1"
RUTA_LINEAMIENTOS = "/content/drive/Shared drives/ACEs/Grabaciones/Lineamientos/Lineamientos.pdf"
ARCHIVO_SALIDA = os.path.join(RUTA_BASE, "RESULTADOS_AUDITORIA_DETALLADA.txt")

client = genai.Client(api_key=API_KEY)

def limpiar_nombre_archivo(nombre):
    """Normaliza el nombre para evitar caracteres que bloqueen el API."""
    nfkd = unicodedata.normalize("NFKD", nombre)
    sin_acentos = "".join([c for c in nfkd if not unicodedata.combining(c)])
    return sin_acentos.replace(" ", "_").replace("(", "").replace(")", "")

def subir_con_diagnostico(ruta_archivo):
    """Sube archivos y espera a que estén activos en el servidor."""
    nombre_original = os.path.basename(ruta_archivo)
    nombre_limpio = limpiar_nombre_archivo(nombre_original)
    print(f"  > [PASO 1] Subiendo: {nombre_original}...")

    try:
        f = client.files.upload(file=ruta_archivo, config={"display_name": nombre_limpio})
        inicio_espera = time.time()
        timeout = 300  # 5 minutos para audios/videos/PDF

        while f.state.name == "PROCESSING":
            if time.time() - inicio_espera > timeout:
                print(f"\n  [!] ALERTA: Timeout al subir {nombre_original}.")
                return None
            time.sleep(5)
            f = client.files.get(name=f.name)

        if f.state.name == "ACTIVE":
            print(f"  > [PASO 2] Archivo activo y listo.")
            # 👇 Aquí imprimimos el MIME type detectado
            print(f"  [INFO] Archivo: {nombre_original} | URI: {f.uri} | MIME: {f.mime_type}")
            return f
        else:
            print(f"\n  [X] ERROR: El archivo quedó en estado: {f.state.name}")
            return None

    except Exception as e:
        print(f"\n  [X] ERROR CRÍTICO durante la subida: {str(e)}")
        print(f"  [DEBUG] Tipo de excepción: {type(e)}")
        print(f"  [DEBUG] Representación completa: {repr(e)}")
        return None


def ejecutar_auditoria():
    print("=== INICIANDO SISTEMA DE AUDITORÍA CON DIAGNÓSTICO ===")

    # 1. Subir PDF de lineamientos
    print("\n1. SUBIENDO LINEAMIENTOS (PDF)...")
    if not os.path.exists(RUTA_LINEAMIENTOS):
        print(f"Error: No se encontró el PDF en {RUTA_LINEAMIENTOS}")
        return

    file_pdf = subir_con_diagnostico(RUTA_LINEAMIENTOS)
    if not file_pdf:
        print("No se pudo continuar sin los lineamientos.")
        return

    # 2. Buscar Archivos de Audio/Video
    print("\n2. BUSCANDO ARCHIVOS MULTIMEDIA...")
    extensiones = ["*.mp4", "*.mp3", "*.wav", "*.WAV", "*.m4a"]
    lista_audios = []
    for ext in extensiones:
        lista_audios.extend(glob.glob(os.path.join(RUTA_BASE, ext)))

    total = len(lista_audios)
    if total == 0:
        print("No se encontraron archivos para auditar.")
        return

    print(f"Total a procesar: {total} archivos.")

    # 3. Prompt de análisis
    prompt_analisis = (
        "Actúa como un auditor de calidad experto. Analiza el audio/video adjunto utilizando "
        "estrictamente los lineamientos del PDF proporcionado. "
        "Genera un reporte con: 1. Resumen de la llamada, 2. Lista de incumplimientos detectados, "
        "3. Calificación numérica (0-100) y 4. Recomendación de mejora."
    )

    with open(ARCHIVO_SALIDA, "w", encoding="utf-8") as f_salida:
        f_salida.write("REPORTES DE AUDITORÍA GENERADOS\n" + "="*30 + "\n")

        for i, ruta in enumerate(lista_audios, 1):
            nombre_file = os.path.basename(ruta)
            print(f"\n[{i}/{total}] Analizando: {nombre_file}")

            file_multimedia = subir_con_diagnostico(ruta)

            if file_multimedia:
                try:
                    print("  > [PASO 3] Generando análisis con Gemini...")
                    response = client.models.generate_content(
                        model=MODELO_PROCESO,
                        contents=[
                            {"file_data": {"file_uri": pdf_file.uri, "mime_type": pdf_file.mime_type}},
                            {"file_data": {"file_uri": file_multimedia.uri, "mime_type": file_multimedia.mime_type}},
                            {"text": prompt_analisis}
                        ]
                    )


                    texto_analisis = response.text
                    f_salida.write(f"\nARCHIVO: {nombre_file}\n")
                    f_salida.write(f"{texto_analisis}\n")
                    f_salida.write("-" * 50 + "\n")
                    f_salida.flush()

                    print("  [OK] Análisis completado con éxito.")

                    # Limpiar archivo multimedia en la nube
                    client.files.delete(name=file_multimedia.name)

                except Exception as e:
                    print(f"  [X] Error en el análisis de Gemini: {str(e)}")
                    print(f"  [DEBUG] Tipo de excepción: {type(e)}")
                    print(f"  [DEBUG] Representación completa: {repr(e)}")
            else:
                print(f"  [!] Saltando archivo {nombre_file} por error en procesamiento.")

    # Limpieza final del PDF
    client.files.delete(name=file_pdf.name)
    print("\n=== PROCESO FINALIZADO ===")
    print(f"Resultados guardados en: {ARCHIVO_SALIDA}")

if __name__ == "__main__":
    ejecutar_auditoria()



=== INICIANDO SISTEMA DE AUDITORÍA CON DIAGNÓSTICO ===

1. SUBIENDO LINEAMIENTOS (PDF)...
  > [PASO 1] Subiendo: Lineamientos.pdf...
  > [PASO 2] Archivo activo y listo.
  [INFO] Archivo: Lineamientos.pdf | URI: https://generativelanguage.googleapis.com/v1beta/files/944wirga3g0n | MIME: application/pdf

2. BUSCANDO ARCHIVOS MULTIMEDIA...
Total a procesar: 4 archivos.

[1/4] Analizando: EMPRESA1_TERCEROS.wav
  > [PASO 1] Subiendo: EMPRESA1_TERCEROS.wav...
  > [PASO 2] Archivo activo y listo.
  [INFO] Archivo: EMPRESA1_TERCEROS.wav | URI: https://generativelanguage.googleapis.com/v1beta/files/z7gpdsv4q73q | MIME: audio/x-wav
  > [PASO 3] Generando análisis con Gemini...
  [OK] Análisis completado con éxito.

[2/4] Analizando: EMPRESA1_RECORDATORIO.wav
  > [PASO 1] Subiendo: EMPRESA1_RECORDATORIO.wav...
  > [PASO 2] Archivo activo y listo.
  [INFO] Archivo: EMPRESA1_RECORDATORIO.wav | URI: https://generativelanguage.googleapis.com/v1beta/files/yh2fx9etfz6d | MIME: audio/x-wav
  > [PASO 3]

In [ ]:
%cd "/content/drive/Shared drives/mi_ruta/Empresa1"

/content/drive/Shared drives/mi_ruta/Empresa1


In [ ]:
import os
import glob
carpeta = "/content/drive/Shared drives/mi_ruta/"
os.chdir(carpeta)
# Extensiones que quieres convertir
extensiones = ["*.mp4", "*.mp3", "*.aac", "*.m4a"]
for ext in extensiones:
  for archivo in glob.glob(ext):
    base = os.path.splitext(archivo)[0]
    salida = base + ".wav" # Ejecuta ffmpeg desde Python
    os.system(f'ffmpeg -y -i "{archivo}" -ar 16000 -ac 1 "{salida}"')
    # Elimina el original
    os.remove(archivo)
    print(f"Convertido y eliminado: {archivo} -> {salida}")


Convertido y eliminado: EMPRESA1_TERCEROS.mp4 -> EMPRESA1_TERCEROS.wav
Convertido y eliminado: EMPRESA1_RECORDATORIO.mp4 -> EMPRESA1_RECORDATORIO.wav
Convertido y eliminado: EMPRESA1_PROMESA.mp4 -> EMPRESA1_PROMESA.wav
Convertido y eliminado: EMPRESA1_NEGATIVA.mp4 -> EMPRESA1_NEGATIVA.wav
